In [ ]:
import numpy as np
import scipy as sp
import scipy.stats as stats
import plotly
import plotly.graph_objs as go

# Import Gaussian process from scikit-learn (https://scikit-learn.org/)
import sklearn.gaussian_process as sklgp
import sklearn.linear_model as skllin

# Import rapid_models DOE packages
#import rapid_models.doe as doe
#import rapid_models.doe.adaptive_learning as doe_al

In [ ]:
def black_box_func(x):
    """A non-linear "black box" function used to 
    illustrate aspects of Design-of-experiments
    
    """
    return np.cos((x+0.05)*2*np.pi)

In [ ]:
x=np.linspace(0,1, 100)
y=black_box_func(x)
s1_x=stats.uniform.rvs(loc=0.1, scale=0.3, size=30, random_state=42)
s1_y = black_box_func(s1_x) + stats.norm.rvs(loc=0, scale=0.05, size=len(s1_x), random_state=42)

s2_x=stats.uniform.rvs(loc=0.5, scale=0.5, size=5, random_state=42)
s2_y = black_box_func(s2_x) + stats.norm.rvs(loc=0, scale=0.05, size=len(s2_x), random_state=42)

In [ ]:
# Plot the result
fig = go.FigureWidget()


fig.add_scatter(x=x, y=y, mode='lines',
                  line={'color':'black'},
                  name='true function', showlegend=True)
fig.add_scatter(x=s1_x, y=s1_y, mode='markers',
                  marker={'symbol':'circle', 'color':'black', 'size':5},
                  name='samples 1', showlegend=True)
fig.add_scatter(x=s2_x, y=s2_y, mode='markers',
                  marker={'symbol':'x', 'color':'black', 'size':5},
                  name='samples 2', showlegend=True)

#fig.update_layout(xaxis={'range':[0,1]},
#                  yaxis={'range':[0,1]}
#                  )

fig.show()

In [ ]:
def train_gp(X_train, y_train, kernel, noise=1e-10, n_restart_optimizer=5, random_state=42):
    """
    Train a scikit-learn gpm
    
    **Parameters:**
    X_train: training input data, array-like of shape (n_samples, n_dimensions)
    y_train: training output data, array-like of shape (n_samples)
    kernel:  GP kernel
    noise:   Interpreted as the variance of additional Gaussian measurement noise on the training observations.
             Value will be added to the diagonal of the kernel matrix
    n_restart_optimizer: Number of restarts of the optimizer
    random_state: Determines random number generation used to initialize the centers. Pass an int for reproducible results across multiple function calls.
    """
   
    gpm = sklgp.GaussianProcessRegressor(alpha=noise, kernel=kernel, n_restarts_optimizer=n_restart_optimizer, random_state=random_state)
    gpm.fit(X_train, y_train)
    return gpm

In [ ]:
length_scale=1/9
kernel = sklgp.kernels.RBF(length_scale=length_scale, length_scale_bounds=(1/10, 1/9))
#kernel = sklgp.kernels.Matern(length_scale=length_scale, nu=5/2,length_scale_bounds=(1/10, 1/9))

In [ ]:
lin_s1 = skllin.LinearRegression()
lin_s1.fit(s1_x.reshape(-1, 1), s1_y)


In [ ]:
gpm_s1 = train_gp(s1_x.reshape(-1, 1), s1_y.reshape(-1, 1), kernel, noise=0.05)
gpm_s2 = train_gp(np.hstack([s1_x,s2_x]).reshape(-1, 1), np.hstack([s1_y,s2_y]).reshape(-1, 1), kernel, noise=0.02)

In [ ]:
# Predict based on fitted GP
mu_s1,std_s1 = gpm_s1.predict(x.reshape(-1,1), return_std=True)
mu_s2,std_s2 = gpm_s2.predict(x.reshape(-1,1), return_std=True)

In [ ]:
s1_ys = gpm_s1.sample_y(x.reshape(-1,1), n_samples=15)
s2_ys = gpm_s2.sample_y(x.reshape(-1,1), n_samples=15)

In [ ]:
# Plot the result
fig = go.FigureWidget()

# fig.add_scatter(x=x, y=y, mode='lines',
#                   line={'color':'black'},
#                   name='true function', showlegend=True)

bgps2=True
blin=False
bs2=True
bGP=True
showlegend=False
if bGP:
  xrange=[0,1]
  if bgps2:
    for q in range(s2_ys.shape[2]):
      showlegend_s=True if (q==0 and showlegend) else False
      fig.add_scatter(x=x, y=s2_ys[:,0,q], mode='lines',
                      line={'color':'red', 'width':0.2},
                      name='GP samples', legendgroup='GP samples', 
                      showlegend=showlegend_s)  
    #fig.add_scatter(x=x, y=mu_s2[:,0], mode='lines',
    #                  line={'color':'black'},
    #                  name='GP mean', showlegend=True)
    fig.add_scatter(x=x, y=mu_s2[:,0]-2*std_s2, mode='lines',
                      line={'color':'lightgray', 'width':0},
                      name='GP-std', showlegend=showlegend)
    fig.add_scatter(x=x, y=mu_s2[:,0]+2*std_s2, mode='lines',
                      line={'color':'lightgray', 'width':0},
                      name='GP+std', showlegend=showlegend, fill='tonexty')
  else:
    for q in range(s2_ys.shape[2]):
      showlegend_s=True if (q==0 and showlegend) else False
      fig.add_scatter(x=x, y=s1_ys[:,0,q], mode='lines',
                      line={'color':'red', 'width':0.2},
                      name='GP samples', legendgroup='GP samples', showlegend=showlegend_s)  
    #fig.add_scatter(x=x, y=mu_s1[:,0], mode='lines',
    #                  line={'color':'black'},
    #                  name='GP mean', showlegend=True)
    fig.add_scatter(x=x, y=mu_s1[:,0]-2*std_s1, mode='lines',
                      line={'color':'lightgray', 'width':0},
                      name='GP-std', showlegend=False)
    fig.add_scatter(x=x, y=mu_s1[:,0]+2*std_s1, mode='lines',
                      line={'color':'lightgray', 'width':0},
                      name='GP+std', showlegend=showlegend, fill='tonexty')       
else:
  xrange=[0,0.5]

if blin:
  fig.add_scatter(x=x, y=lin_s1.predict(x.reshape(-1,1)), mode='lines',
                    line={'color':'black'},
                    name='Linear regression', showlegend=showlegend)
if bs2:
  fig.add_scatter(x=s2_x, y=s2_y, mode='markers',
                    marker={'symbol':'cross', 'color':'black', 'size':5},
                    name='samples 2', showlegend=False)                            
fig.add_scatter(x=s1_x, y=s1_y, mode='markers',
                    marker={'symbol':'circle', 'color':'black', 'size':5},
                    name='samples 1', showlegend=showlegend)

fig.update_layout(width=700, height=500,
                  xaxis={'range':xrange, 'showticklabels':False, 'ticks':'outside', 'linecolor': 'black', 'linewidth': 1, 'mirror': True, 'showgrid':False, 'zeroline':False},
                  yaxis={'range':[-1.2,1.2], 'showticklabels':False, 'ticks':'outside', 'linecolor': 'black', 'linewidth': 1, 'mirror': True, 'showgrid':False, 'zeroline':False},
                  plot_bgcolor='white', paper_bgcolor='white'
                  )

fig.show()